In [1]:
import sys
import os
path_pipest = os.path.abspath('./')
n=0
while (not os.path.basename(path_pipest)=='pipest') and (n<4):
    path_pipest=os.path.dirname(path_pipest)
    n+=1 
if not os.path.basename(path_pipest)=='pipest':
    print("path_pipest not found. Instead: {}".format(path_pipest))
    raise ValueError("path_pipest not found.")
path_models=path_pipest+'/models'    
path_sdhawkes=path_pipest+'/sdhawkes'
path_modelling = path_sdhawkes+'/modelling'
path_resources = path_sdhawkes+'/resources'
path_lobster=path_pipest+'/lobster'
path_lobster_data=path_lobster+'/data'
path_lobster_pyscripts=path_lobster+'/py_scripts'
path_tests = path_pipest+'/tests'
path_saved_tests = path_tests+'/saved_tests'
path_perfmeas=path_tests+'/performance_measurements'
sys.path.append(path_modelling)
sys.path.append(path_resources)
sys.path.append(path_perfmeas)
import numpy as np
import pandas as pd
import pickle
import datetime
import time
import datetime
import timeit
import model as sd_hawkes_model
import plot_tools
import prepare_from_lobster as from_lobster
import lob_model

openmp.omp_get_max_threads(): 4


## Empirical analysis of LOBSTER

In [2]:
symbol, date = 'INTC', '2019-01-23'
initial_time, final_time = float(37800), float(41400)
time_window=str('{}-{}'.format(int(initial_time),int(final_time)))

In [3]:
LOB,messagefile=from_lobster.read_from_LOBSTER(symbol, date,
                 dump_after_reading=False,
                 add_level_to_messagefile=True
                )
LOB,messagefile=from_lobster.select_subset(LOB, messagefile,
                 initial_time=initial_time,
                 final_time=final_time
                )

Given shape of orderbook: 1081948x40


/home/claudio/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


I am dropping column 6 of messagefile. Please disregard any Warning about this column
given shape of messagefile: 1081948x6
read_from_LOBSTER: time window:
  initial_time=34200.018282968
  final_time=57599.983832222
I am adding 'level' to messagefile
declare_level: check = True


In [4]:
print(messagefile.shape)
print(LOB.shape)

(188200, 8)
(188200, 42)


In [5]:
man_mf = from_lobster.ManipulateMessageFile(
        LOB,messagefile, symbol=symbol,date=date,
        aggregate_time_stamp=True,
        eventTypes_to_aggregate=[1,2,3,4,5],
        separate_directions=True,
        equiparate_45_events_with_same_time_stamp=True,
        eventTypes_to_drop_after_aggregation=[3],
        drop_5_events_with_same_time_stamp_as_4=True,
        drop_5_events_after_aggregation=True,
        tolerance_when_dropping=1.0e-8,
        clear_same_time_stamp=True,
        add_hawkes_marks=True,
        num_iter=4
)
man_ob = from_lobster.ManipulateOrderBook(
        man_mf.LOB_sdhawkes,symbol=symbol,date=date,
ticksize = man_mf.ticksize,n_levels=man_mf.n_levels,volume_imbalance_upto_level=2)
data = from_lobster.DataToStore(man_ob,man_mf,time_origin=initial_time)
data.LOB.drop(columns=['original_idx'], inplace=True, errors='ignore')
data.messagefile.drop(columns=['original_idx', 'delta_t', 'same_time_stamp'], inplace=True, errors='ignore')
data.messagefile.columns = [col if col!='hawkes_mark' else 'hawkes_component' for col in data.messagefile.columns]
data.messagefile['hawkes_component']+=1
df_event_encoding = man_mf.event_enc.df_event_enc
df_event_encoding.columns = ['hawkes_component', 'lobster_event_type', 'direction', 'level']
df_event_encoding['hawkes_component']+=1

Initial length of the message file: 188200
ManipulateMessageFile.aggregate_time_stamp: contraction. only_4_events=False
  From 18022 lines with same time stamp, same event type and same direction, -->  to 8578 lines: 9444 lines contracted
  Initial length=188200, contracted length=178756
  Number of remaining events with same time stamp=559
I am dropping events with non-unique time stamp and event type=3, while keeping the_last of the events in every batch
Numbers of events dropped: 17
I am dropping events with non-unique time stamp and event type=5, while keeping the_last of the events in every batch
Numbers of events dropped: 44
Total number of events dropped: 61
prepare_from_lobster.separate_directions: number of occurrencies = 127 
   After separation, the average time step is 0.2320886509389001, the smallest time step is 1.2999953469261527e-07.
ManipulateMessageFile.aggregate_time_stamp: contraction. only_4_events=False
  From 282 lines with same time stamp, same event type and sa

In [6]:
df_event_encoding

,hawkes_component,lobster_event_type,direction,level
0,1,4,1,1
1,1,5,1,1
2,2,4,-1,1
3,2,5,-1,1
4,3,1,-1,1
5,3,1,-1,2
6,3,2,1,1
7,3,2,1,2
8,3,3,1,1
9,3,3,1,2


In [7]:
data.messagefile

,direction,event_type,level,price,size,time,hawkes_component
0,1,3,1,481700,100,37800.007359,3
1,-1,1,2,481900,100,37800.007505,3
2,-1,1,1,481800,100,37800.007568,3
3,-1,1,1,481800,100,37800.007572,3
4,-1,1,1,481800,100,37800.007718,3
...,...,...,...,...,...,...,...
153069,1,1,1,478300,100,41399.124310,4
153070,1,1,1,478300,100,41399.331114,4
153071,-1,1,1,478400,400,41399.595825,3
153072,1,1,1,478300,100,41399.787268,4


In [8]:
# data.messagefile.drop(columns='time').describe().to_latex()

In [9]:
cols = [*data.LOB.columns[0:8], *['time', 'one_dim_state']]
data.LOB.loc[:,cols]

,ask_price_1,ask_volume_1,bid_price_1,bid_volume_1,ask_price_2,ask_volume_2,bid_price_2,bid_volume_2,time,one_dim_state
0,481800,100,481700,700,481900,3500,481600,3900,37800.007359,7
1,481800,100,481700,700,481900,3600,481600,3900,37800.007505,7
2,481800,200,481700,700,481900,3600,481600,3900,37800.007568,7
3,481800,300,481700,700,481900,3600,481600,3900,37800.007572,7
4,481800,400,481700,700,481900,3600,481600,3900,37800.007718,7
...,...,...,...,...,...,...,...,...,...,...
153069,478400,600,478300,1148,478500,3355,478200,2900,41399.124310,7
153070,478400,600,478300,1248,478500,3355,478200,2900,41399.331114,7
153071,478400,1000,478300,1248,478500,3355,478200,2900,41399.595825,7
153072,478400,1000,478300,1348,478500,3355,478200,2900,41399.787268,7


In [10]:
# cols.remove('time')
# data.LOB.loc[:,cols].describe().to_latex()

In [17]:
mf=data.messagefile
lob=data.LOB
idx_pricedecrease=lob['one_dim_state'].isin([0,1,2,3,4])
mf.loc[idx_pricedecrease,:].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).to_latex()

'\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &    direction &   event\\_type &        level &          price &         size &          time &  hawkes\\_component \\\\\n\\midrule\ncount &  1117.000000 &  1117.000000 &  1117.000000 &    1117.000000 &  1117.000000 &   1117.000000 &       1117.000000 \\\\\nmean  &    -0.063563 &     2.306177 &     0.552372 &  480399.641898 &   606.469114 &  39476.397162 &          2.273948 \\\\\nstd   &     0.998425 &     1.414621 &     0.533960 &    1094.512157 &   987.982347 &   1049.765650 &          0.966357 \\\\\nmin   &    -1.000000 &     1.000000 &     0.000000 &  477700.000000 &     1.000000 &  37808.475607 &          1.000000 \\\\\n1\\%    &    -1.000000 &     1.000000 &     0.000000 &  477900.000000 &    28.640000 &  37824.265563 &          1.000000 \\\\\n25\\%   &    -1.000000 &     1.000000 &     0.000000 &  480000.000000 &   100.000000 &  38528.623704 &          1.000000 \\\\\n50\\%   &    -1.000000 &     1.000000 &     1.000000 &  480300.000000

In [18]:
mf.columns

Index(['direction', 'event_type', 'level', 'price', 'size', 'time',
       'hawkes_component'],
      dtype='object')

In [23]:
idx=np.logical_and(idx_pricedecrease, mf['hawkes_component']==1)
mf.loc[idx,['direction', 'event_type', 'level', 'price', 'size','hawkes_component']].describe()

,direction,event_type,level,price,size,hawkes_component
count,407.0,407.0,407.0,407.000000,407.000000,407.0
mean,1.0,4.0,0.0,480334.398034,365.882064,1.0
std,0.0,0.0,0.0,1152.079323,504.227308,0.0
min,1.0,4.0,0.0,477700.000000,1.000000,1.0
25%,1.0,4.0,0.0,479950.000000,100.000000,1.0
50%,1.0,4.0,0.0,480300.000000,200.000000,1.0
75%,1.0,4.0,0.0,481300.000000,400.000000,1.0
max,1.0,4.0,0.0,482400.000000,5740.000000,1.0


In [20]:
idx=np.logical_and(idx_pricedecrease, mf['hawkes_component']==3)
mf.loc[idx,['direction', 'event_type', 'level', 'price', 'size', 'hawkes_component']].describe()

,direction,event_type,level,price,size,hawkes_component
count,707.000000,707.000000,707.000000,707.000000,707.000000,707.0
mean,-0.671853,1.328147,0.864215,480434.087694,741.459689,3.0
std,0.741209,0.741209,0.410494,1059.162429,1154.922432,0.0
min,-1.000000,1.000000,0.000000,477700.000000,4.000000,3.0
25%,-1.000000,1.000000,1.000000,480000.000000,100.000000,3.0
50%,-1.000000,1.000000,1.000000,480400.000000,100.000000,3.0
75%,-1.000000,1.000000,1.000000,481300.000000,400.000000,3.0
max,1.000000,3.000000,2.000000,482500.000000,3400.000000,3.0


In [14]:
idx_priceincrease=lob['one_dim_state'].isin([10,11,12,13,14])
mf.loc[idx_priceincrease,:].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99])

,direction,event_type,level,price,size,time,hawkes_component
count,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000
mean,-0.046729,2.379439,0.473832,480424.018692,618.331776,39450.053753,3.328037
std,0.999375,1.364369,0.499548,1100.604783,1004.339501,1051.194563,0.943621
min,-1.000000,1.000000,0.000000,477700.000000,1.000000,37810.023457,2.000000
1%,-1.000000,1.000000,0.000000,477869.000000,10.690000,37833.111659,2.000000
25%,-1.000000,1.000000,0.000000,480000.000000,100.000000,38483.660590,2.000000
50%,-1.000000,3.000000,0.000000,480400.000000,100.000000,39393.310986,4.000000
75%,1.000000,4.000000,1.000000,481300.000000,400.000000,40322.196182,4.000000
99%,1.000000,4.000000,1.000000,482300.000000,3300.000000,41328.565122,4.000000
max,1.000000,4.000000,1.000000,482500.000000,4351.000000,41360.717292,4.000000


In [15]:
idx=np.logical_and(idx_priceincrease, mf['hawkes_component']==2)
mf.loc[idx,:].describe()

,direction,event_type,level,price,size,time,hawkes_component
count,358.0,358.0,358.0,358.000000,358.000000,358.000000,358.0
mean,-1.0,4.0,0.0,480374.581006,375.385475,39484.302036,2.0
std,0.0,0.0,0.0,1104.762655,502.076218,1045.120003,0.0
min,-1.0,4.0,0.0,477800.000000,4.000000,37826.674339,2.0
25%,-1.0,4.0,0.0,480000.000000,100.000000,38541.632774,2.0
50%,-1.0,4.0,0.0,480300.000000,200.000000,39478.059090,2.0
75%,-1.0,4.0,0.0,481300.000000,400.000000,40327.973291,2.0
max,-1.0,4.0,0.0,482400.000000,4351.000000,41357.912526,2.0


In [16]:
idx=np.logical_and(idx_priceincrease, mf['hawkes_component']==4)
mf.loc[idx,:].describe()

,direction,event_type,level,price,size,time,hawkes_component
count,709.000000,709.000000,709.000000,709.000000,709.000000,709.000000,709.0
mean,0.435825,1.564175,0.713681,480444.710860,741.533145,39432.439754,4.0
std,0.900667,0.900667,0.452359,1097.166784,1161.996068,1054.526095,0.0
min,-1.000000,1.000000,0.000000,477700.000000,1.000000,37810.023457,4.0
25%,-1.000000,1.000000,0.000000,480000.000000,100.000000,38467.001274,4.0
50%,1.000000,1.000000,1.000000,480400.000000,100.000000,39303.054314,4.0
75%,1.000000,3.000000,1.000000,481300.000000,400.000000,40317.514367,4.0
max,1.000000,3.000000,1.000000,482400.000000,3400.000000,41360.717292,4.0
